# Digital Image processing Exercise 3-3

Amirkabir University of Technology

Dr. Rahmati

by Gholamreza Dar 400131018

Spring 2022

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
sns.set_style('dark')

## Functions

In [ ]:
def rgb(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

def bgr(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

def gray(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def disp(img, title='', s=8, vmin=None, vmax=None):
    plt.figure(figsize=(s,s))
    plt.axis('off')
    if vmin is not None and vmax is not None:
        plt.imshow(img, cmap='gray', vmin=vmin, vmax=vmax)
    else:
        plt.imshow(img, cmap='gray')
    plt.title(title)
    plt.show()

In [ ]:
def fourier_analysis(img):
    fourier_img = cv2.dft(img.astype(np.float32), flags=cv2.DFT_COMPLEX_OUTPUT)
    fourier_img_shift = np.fft.fftshift(fourier_img)
    real = fourier_img_shift[:,:,0]
    imag = fourier_img_shift[:,:,1]
    magnitude = cv2.magnitude(real, imag)
    phase = cv2.phase(real, imag)
    return magnitude, phase


In [ ]:
def plot_magnitude_and_phase(img, name, vmin=None, vmax=None):
    # Calculate the Fourier transform of the image
    magnitude, phase = fourier_analysis(img)
    
    # Display the magnitude and phase of the image
    fig, axs = plt.subplots(1,3, figsize=(15,5), constrained_layout=True)
    fig.suptitle(f"'{name}' Fourier Analysis")

    axs[0].set_axis_off()
    axs[1].set_axis_off()
    axs[2].set_axis_off()

    magnitude = np.log(magnitude)

    if vmin is not None and vmax is not None:
        axs[0].imshow(img, cmap='gray')
        axs[1].imshow(magnitude, cmap='gray', vmin=vmin, vmax=vmax)
        axs[2].imshow(phase, cmap='gray')
    else:
        axs[0].imshow(img, cmap='gray')
        axs[1].imshow(magnitude, cmap='gray')
        axs[2].imshow(phase, cmap='gray')
    
    fig.savefig(f'{name}_fourier_analysis.png')
    plt.show()

## Load Images

In [ ]:
images = []
for i in range(1, 8+1):
    images.append(rgb(cv2.imread('inputs/P3/painting_' + str(i) + '.png')))
    

## Tests

In [ ]:
img = gray(images[2])
disp(img)

### Numpy

In [ ]:
%%timeit
fimg = np.fft.fft2(img)
fimg = np.fft.fftshift(fimg)
real = fimg.real
imag = fimg.imag
res = np.log(np.abs(real))
# disp(res, vmin=7, vmax=18)

### Tile the image to see the effect of  fftshift

In [ ]:
fimg = np.fft.fft2(img)
# fimg = np.fft.fftshift(fimg)
real = fimg.real
imag = fimg.imag
res = np.log(np.abs(real))
disp(res, vmin=7, vmax=18)

In [ ]:
fimg_rep = np.tile(real, (2,2))
disp(np.log(np.abs(fimg_rep)), vmin=7, vmax=18)

In [ ]:
fimg = np.fft.fft2(img)
fimg = np.fft.fftshift(fimg)
real = fimg.real
imag = fimg.imag
res = np.log(np.abs(real))
disp(res, vmin=7, vmax=18)

In [ ]:
fimg_rep = np.tile(real, (2,2))
disp(np.log(np.abs(fimg_rep)), vmin=7, vmax=18)

### OpenCV

In [ ]:
%%timeit
fimg = cv2.dft(img.astype(np.float32), flags=cv2.DFT_COMPLEX_OUTPUT)
fimg_shift = np.fft.fftshift(fimg)
real = fimg_shift[:,:,0]
imag = fimg_shift[:,:,1]
# disp(np.log(np.abs(real)), vmin=7, vmax=18)

opencv is faster than numpy (about 3 times)

In [ ]:
img = gray(images[2])
disp(img)

In [ ]:
magnitude, phase = fourier_analysis(img)
disp(np.log(magnitude), vmin=7, vmax=18)

# Main

In [ ]:
for i, img in enumerate(images):
    # img = gray(img)
    labels = list("RGB")
    for channel in range(3):
        plot_magnitude_and_phase(img[:, :, channel], f'painting_{i}_{labels[channel]}', vmin=8, vmax=18)

Test inverse phase


In [ ]:
img = images[0]
img = gray(img)
fourier_img = cv2.dft(img.astype(np.float32), flags=cv2.DFT_COMPLEX_OUTPUT)
fourier_img_shift = np.fft.fftshift(fourier_img)

real = fourier_img_shift[:,:,0]
imag = fourier_img_shift[:,:,1]

magnitude = cv2.magnitude(real, imag)
phase = cv2.phase(real, imag)

# MERGE magnitude and phase

inv_shift = np.fft.ifftshift(fourier_img_shift)
result = cv2.idft(inv_shift, flags=cv2.DFT_SCALE)

filtered_img = np.zeros((img.shape[0], img.shape[1]), dtype=np.float32)
filtered_img[:,:] = cv2.magnitude(result[:,:,0], result[:,:,1])

disp(filtered_img)
